In [ ]:
import copy, re

In [ ]:
from nltk.corpus import semcor
import nltk
import numpy

nltk.download("semcor")
nltk.download('wordnet')

[nltk_data] Downloading package semcor to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.corpus import wordnet as wn

In [ ]:
help(semcor.tagged_sents)

Help on method tagged_sents in module nltk.corpus.reader.semcor:

tagged_sents(fileids=None, tag='pos') method of nltk.corpus.reader.semcor.SemcorCorpusReader instance
    :return: the given file(s) as a list of sentences. Each sentence
        is represented as a list of tagged chunks (in tree form).
    :rtype: list(list(Tree))
    
    :param tag: `'pos'` (part of speech), `'sem'` (semantic), or `'both'`
        to indicate the kind of tags to include.  Semantic tags consist of
        WordNet lemma IDs, plus an `'NE'` node if the chunk is a named entity
        without a specific entry in WordNet.  (Named entities of type 'other'
        have no lemma.  Other chunks not in WordNet have no semantic tag.
        Punctuation tokens have `None` for their part of speech tag.)



In [ ]:
input_seq="Jacob wants to immerse himself in some books."

In [ ]:
from nltk.corpus import semcor
tagged_sents = [[str(c) for c in s] for s in semcor.tagged_sents(tag='sem')]
tagged_sents_actual = semcor.tagged_sents(tag='sem')

In [ ]:
print(tagged_sents[2])
tagged_sents[2][1]

["['The']", '(september.n.01 September)', '(october.n.01 October)', '(term.n.02 term)', '(jury.n.01 jury)', "['had']", "['been']", '(charge.v.05 charged)', "['by']", '(location.n.01 (NE Fulton))', '(person.n.01 (NE Superior Court Judge Durwood Pye))', "['to']", '(investigate.v.02 investigate)', '(report.n.03 reports)', "['of']", '(possible.a.02 possible)', "['``']", '(irregularity.n.01 irregularities)', '["\'\'"]', "['in']", "['the']", '(hard-fought.s.01 hard-fought)', '(primary.n.01 primary)', "['which']", "['was']", '(win.v.01 won)', "['by']", '(person.n.01 (NE Mayor-nominate Ivan Allen Jr.))', "['.']"]


'(september.n.01 September)'

In [ ]:
tagged_sents[2]
tagged_dataset=[]
for sent in tagged_sents:
    my_arr=[]
    for word in sent:
        if word[0]=="[":
            my_arr.append([word.replace("[","").replace("]","").replace("\'",""),"None"])
        elif word[0]=="(":
            synset=word.split(" ")[0][1:]
            chunk=(" ").join(word.split(" ")[1:]).strip()
            if chunk[0]=="(":
                chunk=chunk[1:-2].strip()
                if chunk[:2]=="NE":
                    chunk=chunk[2:].strip()
                else:
                    print("found other tag ", chunk[:5])
            else:
                chunk=chunk[:-1].strip()
            my_arr.append([chunk,synset])
                
        else:
            print("error ",word)
    tagged_dataset.append(my_arr)

In [ ]:
tagged_dataset[0]

[['The', 'None'],
 ['Fulton County Grand Jury', 'group.n.01'],
 ['said', 'say.v.01'],
 ['Friday', 'friday.n.01'],
 ['an', 'None'],
 ['investigation', 'investigation.n.01'],
 ['of', 'None'],
 ['Atlanta', 'atlanta.n.01'],
 ['"s"', 'None'],
 ['recent', 'recent.s.02'],
 ['primary election', 'primary_election.n.01'],
 ['produced', 'produce.v.04'],
 ['``', 'None'],
 ['no', 'None'],
 ['evidence', 'evidence.n.01'],
 ['""', 'None'],
 ['that', 'None'],
 ['any', 'None'],
 ['irregularities', 'irregularity.n.01'],
 ['took place', 'take_place.v.01'],
 ['.', 'None']]

In [ ]:
MFS = {}
for sent in tagged_dataset:
    for pair in sent:
        chunk = pair[0]
        sense = pair[1]
        if chunk not in MFS:
            MFS[chunk] = {sense:1}
        else:
            if sense not in MFS[chunk]:
                MFS[chunk][sense] = 1
            else:
                MFS[chunk][sense] += 1



for chunk in MFS:
    MFS[chunk] = sorted(MFS[chunk], key = lambda x: MFS[chunk][x], reverse = True)[0]

In [ ]:
untagged_dataset=[]
for sent in semcor.chunk_sents():
    my_arr=[]
    for chunk in sent:
        if len(chunk)==1:
            my_arr.append(chunk)
        else:
            my_arr.append((" ").join(chunk).strip())
    untagged_dataset.append(my_arr)

In [ ]:
def most_freq_sense(index):
    chunked = untagged_dataset[int(index)] #Gradio takes text
    d = []
    for word in chunked:
        if type(word) == list:
            word = word[0]
        
        if word in MFS:
            sense = MFS[word]
        else:
            continue
        
        try:
            if sense.split('.')[1] == 'n':
                d.append(sense)
        except:
            pass
    return ', '.join(d) #Output senses

In [ ]:
untagged_dataset[0]

[['The'],
 'Fulton County Grand Jury',
 ['said'],
 ['Friday'],
 ['an'],
 ['investigation'],
 ['of'],
 ['Atlanta'],
 ["'s"],
 ['recent'],
 'primary election',
 ['produced'],
 ['``'],
 ['no'],
 ['evidence'],
 ["''"],
 ['that'],
 ['any'],
 ['irregularities'],
 'took place',
 ['.']]

In [ ]:
most_freq_sense(0)

'group.n.01, primary_election.n.01, evidence.n.01, irregularity.n.01'

In [ ]:
exp = []
act = []

In [ ]:

for sent in tagged_dataset:
    for pair in sent:
        chunk = pair[0]
        sense = pair[1]
        if len(sense.split('.'))<2:
            continue
        if sense.split('.')[1] == 'n':
            exp.append(MFS[chunk])
            act.append(sense)

In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=770a13f089961f7c4f9dd70d1dd74822413e94b55710aa8e8be90839c155a7cd
  Stored in directory: /root/.cache/pip/wheels/42/56/cc/4a8bf86613aafd5b7f1b310477667c1fca5c51c3ae4124a003
Successfully built sklearn


In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score
print(f1_score(act, exp, average='micro'))
print(recall_score(act, exp, average='micro'))

0.5353112962058112
0.5353112962058112


In [ ]:
exp_wfs = []
act_wfs = []

In [ ]:
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
wn.synsets('none')

[Synset('none.n.01'),
 Synset('none.n.02'),
 Synset('none.s.01'),
 Synset('none.r.01')]

In [ ]:
for sent in tagged_dataset:
    for pair in sent:
        chunk = pair[0]
        sense = pair[1]
        if len(sense.split('.'))<2:
            continue
        if sense.split('.')[1] == 'n':
            wfs = wn.synsets(chunk)
            if len(wfs) == 0:
                continue
            exp_wfs.append(wfs[0].name())
            act_wfs.append(sense)

In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score
print(f1_score(act_wfs, exp_wfs, average='micro'))
print(recall_score(act_wfs, exp_wfs, average='micro'))

0.5624951134971723
0.5624951134971723


In [ ]:
from nltk.corpus import wordnet as wn
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
type(wn.synsets('primary'))

list

In [ ]:
!pip install gradio
import gradio as gr

In [ ]:
print('Most Frequent Sense')

demo = gr.Interface(fn=most_freq_sense, inputs="text", outputs="text")
demo.launch() 

Most Frequent Sense
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
def show(sent):
    nouns = []
    for word in sent.split():
        try:
            if MFS[word].split('.')[1] == 'n':
                nouns.append(MFS[word])
        except:
            pass
    return ', '.join(nouns)

In [ ]:
show("This boy was none other than him")

'boy.n.01'

In [ ]:
print('Most Frequent Sense')

demo = gr.Interface(fn=show, inputs="text", outputs="text")
demo.launch() 

Most Frequent Sense
Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>